In [1]:
import argparse
import DocDictionaryTest as dd
import TextTransformTest as tt
import json
import pandas as pd
import numpy as np
from myGenerator import TestGenerator
from keras.models import load_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Melesio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Melesio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Melesio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Melesio\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Melesio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:

def spectraLoader(inputPath: str) -> pd.DataFrame:
    df = pd.DataFrame()
    with open(inputPath, encoding='utf-8') as f:
        for line in f:
            jsonline = json.loads(line)
            lineDf = pd.DataFrame({'id':jsonline['id'], 'spectra':[np.array(jsonline['spectra'])]})
            df = pd.concat([df, lineDf])
    df = df.reset_index(drop=True)

    return df

In [4]:
print("Leyendo archivo de configuracion")
with open('config.json') as f:
    cfg = json.loads(f.read())

# parser = argparse.ArgumentParser(description='Siamese network AA@PAN')
# parser.add_argument('-i', type=str,
#     help='Path to the input directory')
# parser.add_argument('-o', type=str,
#     help='Path to the output directory')

# args = parser.parse_args()
# args.i = "testinput"
# args.o = "testoutput"
# if not args.i:
#     raise ValueError('The input directory is required')
# if not args.o:
#     raise ValueError('The output directory is required')

# Transformacion de texto

print("Inicia la transformacion de texto")
print("Creando diccionario de documentos unicos")
# dd.createDocDictionary(args.i + '/pairs.jsonl', cfg['tmp'])
dd.createDocDictionary('testinput/pairs.jsonl', cfg['tmp'])

print("Realizando preprocesamiento")
tt.preProcessStage(cfg['tmp'] + '/PanUniqueDocs.jsonl', cfg['tmp'] + '/panPreprocessed.jsonl')

print("Extrayendo vectores de caracteristicas")
tt.featureExtractionStage(cfg['tmp'] + '/panPreprocessed_lex.jsonl',
                            cfg['tmp'] + '/LexicVectors.jsonl',
                            cfg['LexicVectorizer'],
                            cfg['tmp'] + '/panPreprocessed_syn.jsonl',
                            cfg['tmp'] + '/SyntacticVectors.jsonl',
                            cfg['SyntacticVectorizer'],
                            cfg['tmp'] + '/panPreprocessed_sem.jsonl',
                            cfg['tmp'] + '/SemanticVectors.jsonl',
                            cfg['SemanticVectorizer'])

print("Obteniendo espectros de contenido")
tt.spaceMappingStage(cfg['tmp'] + '/LexicVectors.jsonl',
                        cfg['tmp'] + '/LexicSpectra.jsonl',
                        cfg['LexicProjector'],
                        cfg['tmp'] + '/SyntacticVectors.jsonl',
                        cfg['tmp'] + '/SyntacticSpectra.jsonl',
                        cfg['SyntacticProjector'],
                        cfg['tmp'] + '/SemanticVectors.jsonl',
                        cfg['tmp'] + '/SemanticSpectra.jsonl',
                        cfg['SemanticProjector'])

print("Consolidando capas")
tt.layerConsolidation(cfg['tmp'] + '/LexicSpectra.jsonl',
                        cfg['tmp'] + '/SyntacticSpectra.jsonl',
                        cfg['tmp'] + '/SemanticSpectra.jsonl',
                        cfg['tmp'] + '/FullSpectra.jsonl')

Leyendo archivo de configuracion
Inicia la transformacion de texto
Creando diccionario de documentos unicos
Realizando preprocesamiento
Etapa de pre-procesmiento
Extrayendo vectores de caracteristicas
Etapa de extraccion de caracteristicas
Extraccion de vectores lexicos
Realizando vectorizacion
Guardando resultadosto 79
Extraccion de vectores sintacticos
Realizando vectorizacion
Guardando resultados
Extraccion de vectores semanticos
Realizando vectorizacion
Guardando resultados
Obteniendo espectros de contenido
Etapa de unificacion del espacio


c:\Users\Melesio\Documents\SpectralTextRepresentation\minisom\minisom.py:167: UserWarning: Warning: sigma is too high for the dimension of the map.
  warn('Warning: sigma is too high for the dimension of the map.')


Leyendo vectores lexicos
Obteninendo espectros
Leyendo vectores sintacticos
Obteninendo espectros
Leyendo vectores semanticos
Obteninendo espectros
Consolidando capas
Etapa de consolidacion de capas
Información guardada
Ensamble terminado


In [5]:
# Clasificacion
print("Realizando predicciones")

params = {'dim': (1200,),
        'batch_size': 32}

test = pd.read_pickle("tmp/PanTest.plk")
full_spectra = spectraLoader(cfg['tmp'] + '/FullSpectra.jsonl')
list_IDs = test.id.unique().tolist()

Realizando predicciones


In [7]:
x = TestGenerator(list_IDs, test, full_spectra)
model = load_model(cfg['Classifier'])

KeyboardInterrupt: 

In [ ]:
predictions = model.predict(x)

In [ ]:
with open("testoutput/answers.jsonl", "w", encoding="utf8") as f:
    for id, prediction in zip(list_IDs, predictions):
        answer = {"id": id, "value":prediction}
        f.write(json.dumps(answer) + "\n")